In [1]:
%cd ..

/workspace/shai_hulud/ppc_experiments


In [2]:
import argparse
import collections
import lightning
import numpy as np
import pyro
import torch
import logger, train

Seed set to 123


In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
config, (data, model, trainer) = train.from_file("experiments/ppc_flowers_config.json")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, data, ckpt_path="saved/models/FlowersDiffusion_Ppc/0509_231539/checkpoint_999.ckpt")

[rank: 0] Seed set to 123
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
[rank: 1] Seed set to 123
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
Missing logger folder: saved/log/FlowersDiffusion_Ppc/0512_191330/lightning_logs
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

Missing logger folder: saved/log/FlowersDiffusion_Ppc/0512_191330/lightning_logs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /workspace/shai_hulud/ppc_experiments/saved/models/FlowersDiffusion_Ppc/0512_191330 exists and is not empty.
Restoring states from the checkpoint path at saved/models/FlowersDiffusion_Ppc/0509_231539/checkpoint_999.ckpt
[DSI-DGX01:3515055:0:3515055] Caught signal 11 (Segm

ProcessExitedException: process 0 terminated with signal SIGSEGV

In [ ]:
trainer.model.eval()
trainer.cpu()
trainer.valid_particles.cpu()

In [ ]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.nodes[site]['is_observed'] = trainer.model.graph.nodes[site]['value'] is not None

In [ ]:
xs, _, indices = list(trainer.valid_data_loader)[0]
trainer._load_particles(indices, False)
with pyro.plate_stack("forward", (trainer.num_particles, len(xs))):
    model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
    x_hats = model().mean(dim=0)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=8, sharex="all", sharey="all", layout="compressed")

for i in range(8):
    orgs = data_loader.reverse_transform(xs[i].squeeze().detach())
    estimates = data_loader.reverse_transform(x_hats[i].squeeze().detach())
    axes[0, i].imshow(orgs)
    axes[1, i].imshow(estimates)

fig.savefig("ppc_flowers_recons.pdf", bbox_inches="tight")
plt.show()

In [ ]:
del xs
del x_hats

In [ ]:
for site in trainer.model.graph.nodes:
    trainer.model.graph.unclamp(site)

In [ ]:
with pyro.plate_stack("forward", (trainer.num_particles, 8)):
    x_hats = trainer.model()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=8, sharex="all", sharey="all", layout="compressed")

for i in range(8):
    estimates = data_loader.reverse_transform(x_hats[i].squeeze().detach())
    axes[i].imshow(estimates)

fig.savefig("ppc_flowers_samples.pdf", bbox_inches="tight")
plt.show()

In [ ]:
del x_hats

In [ ]:
SEEDS = [123, 456, 789, 101112, 131415]

In [ ]:
LOG_LIKELIHOODS = torch.zeros(len(SEEDS), requires_grad=False)
MEAN_SQUARED_ERROR = torch.zeros(len(SEEDS), requires_grad=False)

In [ ]:
for (s, SEED) in enumerate(SEEDS):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    for b, (xs, target, indices) in enumerate(trainer.valid_data_loader):
        trainer._load_particles(indices, False)
    
        with pyro.plate_stack("forward", (trainer.num_particles, len(xs))):
            model = pyro.condition(trainer.model, data={k: v['value'] for k, v in trainer.model.graph.nodes.items()})
            trace = pyro.poutine.trace(model).get_trace()
            x_hats = trace.nodes['X__0']['value'].mean(dim=0)
        LOG_LIKELIHOODS[s] += trace.nodes['X__0']['fn'].log_prob(xs).sum()
        MEAN_SQUARED_ERROR[s] += ((xs - x_hats) ** 2).sum(dim=0).mean()

        del xs
        del x_hats
        logger.info("Evaluated likelihood for valid batch %d under seed %s" % (b, s))

    LOG_LIKELIHOODS[s] /= len(trainer.valid_data_loader.sampler)
    MEAN_SQUARED_ERROR[s] /= len(trainer.valid_data_loader.sampler)

In [ ]:
LOG_LIKELIHOODS.mean(), LOG_LIKELIHOODS.std()

In [ ]:
MEAN_SQUARED_ERROR.mean(), MEAN_SQUARED_ERROR.std()